In [134]:
import numpy as np
import random
from math import *
import cmath
from scipy.linalg import expm, fractional_matrix_power

#########################################################################

# pauli matrix 
sx = np.array([[0,  1],     [1, 0]])
sy = np.array([[0, -1j],   [1j, 0]])
sz = np.array([[1, 0],     [0, -1]])
s0 = np.array([[1, 0],      [0, 1]])

# parameters(detuning factor)
v0 = 0.02           # Arbitrary settings, Actual speed : 0.04rad/μs
d0 = 0.15           # Arbitrary settings, Actual speed : 0.30rad/μs
dt = 2.6 


# unitary operator
def unitary(dt, choice):
    
    # Select x,y-rotation direction.
    # [stay, +x, -x, +y, -y]
    choice_list = [0, 1, -1, 1, -1] 
    
    if choice < 3:
        # if choice = 0 ... only d0*sz
        Ham = (d0*sz+v0*choice_list[choice]*sx)
    else:
        Ham = (d0*sz+v0*choice_list[choice]*sy)

    # Creating a Unitary Operator for each of the four sections by Hamiltonian
    eigvals = np.linalg.eigh(Ham)[0]
    eigvecs = 1*np.linalg.eigh(Ham)[1]
    E = np.diag(eigvals)
    U_H = eigvecs.conj().T
    U_e = U_H.conj().T @ expm(-1j*E*dt) @ U_H
    
    return U_e


#########################################################################

# x-rotation operater
def Rx(theta):
    return np.matrix([  [cos(theta/2),    -1j*sin(theta/2)],
                        [-1j*sin(theta/2),    cos(theta/2)]])

# z-rotation operater
# Do not use Rz. Control by rotation only by Hamiltonian.
def Rz(phi): 
    return np.matrix([  [cos(phi/2)-1j*sin(phi/2),  0],
                        [0,  cos(phi/2)+1j*sin(phi/2)]])

# Calculating the Fidelity
def state_fidelity(rho_1, rho_2): 
    
    # rho_1(current state), rho_2(target state)
    # Calculate the fidelity after checking the dimensions of the two states.
    
    if np.shape(rho_1) != np.shape(rho_2):
        print("Dimensions of two states do not match.")
        return 0
    else:
        sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
        fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
        return np.real(fidelity)


#########################################################################

init_wave = np.array([[1], [0]])
irho_init = np.kron(init_wave,init_wave.conj().T)

U_0 = unitary(dt, 0)
U_1 = unitary(dt, 1)
U_2 = unitary(dt, 2)
U_3 = unitary(dt, 3)
U_4 = unitary(dt, 4)

pulse_list = [U_0, U_1, U_2, U_3, U_4]


def fidelity(rho, predicted_seq):

    # predicted_seq = np.argmax(predicted_seq, axis=-1)

    print(predicted_seq, type(predicted_seq))
    Uni = s0
    
    for i in predicted_seq:
        Uni = pulse_list[i] @ Uni

    irho_final = Uni @ irho_init @ Uni.conj().T
    
    F = (state_fidelity(irho_final, rho))
    
    return  F

In [135]:
# global input_rho

def find_best_seq(index, seq):
     
     global input_rho

     # return when index is the first element of seq
     if (-index) == len(seq):
          print("Sequence failed because the first element must be changed")
          return temp_seq
     
     temp_seq = []
     temp_fid = 0
     
     # index < 0
     # front_seq = seq[:index]
     # back_seq = seq[index+1:]
     final_seq = seq
   
     if index == -1:                    # there is no back_seq
          for i in range(5):            # 0 to 4 try all elements
               final_seq[index] = i
               
               fid = fidelity(input_rho, final_seq)
               if fid > 99.99:          # return to condition satisfaction
                    return final_seq
               elif fid > temp_fid:     # 
                    temp_fid = fid
                    temp_seq = final_seq
               else:
                    temp_seq = seq
     else:
          for i in range(5):            # 0 to 4 try all elements
               final_seq[index] = i
               final_seq = find_best_seq(index - 1, final_seq)      # recursive function
               
               fid = fidelity(input_rho, final_seq)
               if fid > 99.99:          # return to condition satisfaction
                    return final_seq
               elif fid > temp_fid:
                    temp_fid = fid
                    temp_seq = final_seq
               else:
                    temp_seq = seq

     return temp_seq

In [136]:
import time

def local_optimizer(input_rho, input_seq):

     first_fid = fidelity(input_rho, input_seq)

     if first_fid > 99.99:
          output_seq = input_seq
     else:
          start_time = time.time()
          
          output_seq = find_best_seq(-1, input_seq)
          
          end_time = time.time()
          computing_t = end_time - start_time

     last_fid = fidelity(input_rho, output_seq)

     total_t = dt * len(output_seq)

     return output_seq, first_fid, last_fid, computing_t, total_t



In [137]:
import pandas as pd

df = pd.read_csv('../Data/using/dt_2.6/ByAstar_dt_2.6_1016.csv')
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# df_shuffled.head()

thetas = df_shuffled['Theta']
phis = df_shuffled['Phi']
lists = df_shuffled['combination']

In [138]:

for input_theta, input_phi, input_sequence in zip(thetas, phis, lists):  

     target_U = Rz(input_phi) @ Rx(input_theta)
     input_rho = target_U @ irho_init @ target_U.conj().T
     
     output_sequence, first_fidelity, last_fidelity, computing_time, total_time = local_optimizer(input_rho, eval(input_sequence))

     # Result output
     print(f"""
     ------------------------------------------------------------------------------------------------------
     theta = {input_theta}
     phi = {input_phi}
     input_sequence : {input_sequence}
     first_fidelity : {first_fidelity}
     output_sequence : {output_sequence}
     last_fidelity : {last_fidelity}
     computing_time : {computing_time}
     total_time : {total_time}
     ------------------------------------------------------------------------------------------------------
     """)


[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 2] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 0] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 1] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 2] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 3] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 4] <class 'list'>
[3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 4] <class 'list'>

     ------------------------------------------------------------------------------------------------------
     theta = 1.254180389
     phi = 2.447357318
     input_sequence : [3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 2]
     first_fidelity : 0.9978976173403461
     output_sequence : [3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 2, 1, 3, 4, 4]
     last_fidelity : 0.9928825507464436
     computing_time : 0.015087366104125977
     total_time : 46.800000000000004
     --------------------------------

IndexError: list assignment index out of range